# **615. Average Salary: Departments VS Company**
``` console
Table: Salary

+-------------+------+
| Column Name | Type |
+-------------+------+
| id          | int  |
| employee_id | int  |
| amount      | int  |
| pay_date    | date |
+-------------+------+
In SQL, id is the primary key column for this table.
Each row of this table indicates the salary of an employee in one month.
employee_id is a foreign key (reference column) from the Employee table.
 

Table: Employee

+---------------+------+
| Column Name   | Type |
+---------------+------+
| employee_id   | int  |
| department_id | int  |
+---------------+------+
In SQL, employee_id is the primary key column for this table.
Each row of this table indicates the department of an employee.
 

Find the comparison result (higher/lower/same) of the average salary of employees in a department to the company's average salary.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Salary table:
+----+-------------+--------+------------+
| id | employee_id | amount | pay_date   |
+----+-------------+--------+------------+
| 1  | 1           | 9000   | 2017/03/31 |
| 2  | 2           | 6000   | 2017/03/31 |
| 3  | 3           | 10000  | 2017/03/31 |
| 4  | 1           | 7000   | 2017/02/28 |
| 5  | 2           | 6000   | 2017/02/28 |
| 6  | 3           | 8000   | 2017/02/28 |
+----+-------------+--------+------------+

Employee table:
+-------------+---------------+
| employee_id | department_id |
+-------------+---------------+
| 1           | 1             |
| 2           | 2             |
| 3           | 2             |
+-------------+---------------+

Output: 
+-----------+---------------+------------+
| pay_month | department_id | comparison |
+-----------+---------------+------------+
| 2017-02   | 1             | same       |
| 2017-03   | 1             | higher     |
| 2017-02   | 2             | same       |
| 2017-03   | 2             | lower      |
+-----------+---------------+------------+

Explanation: 
In March, the company's average salary is (9000+6000+10000)/3 = 8333.33...
The average salary for department '1' is 9000, which is the salary of employee_id '1' since there is only one employee in this department. So the comparison result is 'higher' since 9000 > 8333.33 obviously.
The average salary of department '2' is (6000 + 10000)/2 = 8000, which is the average of employee_id '2' and '3'. So the comparison result is 'lower' since 8000 < 8333.33.
```

**Solution-1:**

``` sql
SELECT DISTINCT pay_month, department_id,
CASE 
WHEN AVG(amount) OVER(PARTITION BY pay_month, department_id) > AVG(amount) OVER(PARTITION BY pay_month) THEN 'higher'
WHEN AVG(amount) OVER(PARTITION BY pay_month, department_id) = AVG(amount) OVER(PARTITION BY pay_month) THEN 'same'
WHEN AVG(amount) OVER(PARTITION BY pay_month, department_id) < AVG(amount) OVER(PARTITION BY pay_month) THEN 'lower'
END AS comparison
FROM
(SELECT s.amount, 
CONCAT(SUBSTRING(s.pay_date,1,4), '-', SUBSTRING(s.pay_date,6,2)) AS pay_month, 
e.department_id
FROM Salary s
JOIN Employee e
ON s.employee_id = e.employee_id) a

**Solution-2:**

``` sql
/* Write your T-SQL query statement below */

SELECT DISTINCT pay_month, department_id, 
(
    CASE
    WHEN avg_dept_sal > avg_company_sal THEN "higher"
    WHEN avg_dept_sal < avg_company_sal THEN "lower"
    WHEN avg_dept_sal = avg_company_sal THEN "same"
    END
) as comparison
FROM 
(
    SELECT s.employee_id, amount, pay_Date, department_id, LEFT(s.pay_date, 7) as pay_month,
     AVG(s.amount) OVER(PARTITION BY s.pay_date) as avg_company_sal,
     AVG(s.amount) OVER(PARTITION BY s.pay_Date, e.department_id) as avg_dept_sal
    FROM Salary s
    JOIN Employee e ON s.employee_id = e.employee_id
) as tmp;

**Solution-3:**

``` sql
-- Write your PostgreSQL query statement below
SELECT DISTINCT
TO_CHAR(date_trunc('month', pay_date), 'YYYY-MM') AS pay_month,
department_id,
CASE 
WHEN AVG(amount) OVER (PARTITION BY department_id, date_trunc('month', pay_date)) < AVG(amount) OVER (PARTITION BY date_trunc('month', pay_date)) THEN 'lower' 
WHEN AVG(amount) OVER (PARTITION BY department_id, date_trunc('month', pay_date)) > AVG(amount) OVER (PARTITION BY date_trunc('month', pay_date)) THEN 'higher'
ELSE 'same' END AS comparison
FROM salary s
JOIN employee e
ON s.employee_id = e.employee_id